In [ ]:
from pymongo import MongoClient

Creating MongoDb

In [ ]:
# Client connects to localhost by default
client = MongoClient()

# Create local 'Nobel' database on the fly
db = client['nobel']

documents = response.json()['collection_name']

# Create collections on the fly
db['collection_name'].insert_many(documents)

Accessing data from mongodb

In [ ]:
# One way access data from databaase as dictionareis
# Client is a dictionary of databases
db = client['nobel']

# database is a dictionary of collections
prizes_colection = db['prizes']

# Other way is dot notation
# Databases are an atribute of client
db = client.nobel

# collections are an atribute of database
prizes_colection = db.prizes

Count Documents

In [ ]:
# Use empty document as filter
filter = {}

# Count documents in a collection
n_prizes = db.prizes.count_documents(filter)
n_laureates = db.laureates.count_documents(filter)

Fetch documents

In [ ]:
# List all databases managed by client
client.list_database_names()

# List all collections inside database
db.list_collection_names()

# Find one document to inspect
db.prizes.find_one(filter)

# Get all fields inside document
db.prizes.find_one().keys()

Find Documents

In [ ]:
# Discover how many laureates are female
filter_document = {'gender':'female'}
db.laureates.count_documents(filter_document)

# Query Operators 

# $in: <list>
# Discover how many died in France OR USA
filter_document = {'diedCountry' : {'$in': ['France','USA']}}
db.laureates.count_documents(filter_document)

# $ne: <value> Not equal
filter_document = {'diedCountry' : {'$ne': 'France'}}
db.laureates.count_documents(filter_document)

# $gt: <value> greater than 
# $gte: <value> greater than and equal
# $lt: <value> less than
# $lte: <value> less than and equal
filter_document = {'diedCountry' : {'$gt': 'USA' , '$lte' : 'Belgium'}}
db.laureates.count_documents(filter_document)


# Example
# Save a filter for laureates who died in the USA and were not born there
criteria = { 'diedCountry': 'USA',
             'bornCountry': { "$ne": 'USA'}, 
             }
count = db.laureates.count_documents(criteria)
print(count)

Dot notation: reach into substructure

In [ ]:
# dot notation lets you work in density
criteria = {'prizes.affiliations.name' : ('University of California')}
db.laureates.count_documents(criteria)

Some fields in mongoDb doesnt have to be present

In [ ]:
# Check all laureates that dont have bornCountry
criteria = {'bornCountry' : {'$exists' : False}}
db.laureates.count_documents(criteria)

# Check all laureates that have more than one prize
criteria = {'prizes.1' : {'$exists' : True}}
db.laureates.count_documents(criteria)